# Decorators


Decorators can be thought of as functions which modify the *functionality* of another function. They help to make your code shorter and more "Pythonic". 

To properly explain decorators we will slowly build up from functions. Make sure to restart the Python and the Notebooks for this lecture to look the same on your own computer. So lets break down the steps:

## Functions Review

In [1]:
def func():
    return 1

In [2]:
func()

1

## Scope Review
Remember from the nested statements lecture that Python uses Scope to know what a label is referring to. For example:

In [3]:
s = 'Global Variable'

def func():
    print(locals())

Remember that Python functions create a new scope, meaning the function has its own namespace to find variable names when they are mentioned within the function. We can check for local variables and global variables with the local() and globals() functions. For example:

In [4]:
print(globals())

{'__name__': '__main__', '__doc__': 'Automatically created module for IPython interactive environment', '__package__': None, '__loader__': None, '__spec__': None, '__builtin__': <module 'builtins' (built-in)>, '__builtins__': <module 'builtins' (built-in)>, '_ih': ['', 'def func():\n    return 1', 'func()', "s = 'Global Variable'\n\ndef func():\n    print(locals())", 'print(globals())'], '_oh': {2: 1}, '_dh': ['/Users/mcchae/work/GitHub/python-bootcamp/13-파이썬 데코레이터'], 'In': ['', 'def func():\n    return 1', 'func()', "s = 'Global Variable'\n\ndef func():\n    print(locals())", 'print(globals())'], 'Out': {2: 1}, 'get_ipython': <bound method InteractiveShell.get_ipython of <ipykernel.zmqshell.ZMQInteractiveShell object at 0x108cf1a90>>, 'exit': <IPython.core.autocall.ZMQExitAutocall object at 0x108d3fe10>, 'quit': <IPython.core.autocall.ZMQExitAutocall object at 0x108d3fe10>, '_': 1, '__': '', '___': '', '_i': "s = 'Global Variable'\n\ndef func():\n    print(locals())", '_ii': 'func()',

Here we get back a dictionary of all the global variables, many of them are predefined in Python. So let's go ahead and look at the keys:

In [5]:
print(globals().keys())

dict_keys(['__name__', '__doc__', '__package__', '__loader__', '__spec__', '__builtin__', '__builtins__', '_ih', '_oh', '_dh', 'In', 'Out', 'get_ipython', 'exit', 'quit', '_', '__', '___', '_i', '_ii', '_iii', '_i1', 'func', '_i2', '_2', '_i3', 's', '_i4', '_i5'])


Note how **s** is there, the Global Variable we defined as a string:

In [6]:
globals()['s']

'Global Variable'

Now lets run our function to check for any local variables in the func() (there shouldn't be any)

In [7]:
func()

{}


Great! Now lets continue with building out the logic of what a decorator is. Remember that in Python **everything is an object**. That means functions are objects which can be assigned labels and passed into other functions. Lets start with some simple examples:

In [8]:
def hello(name='Jose'):
    return 'Hello '+name

In [9]:
hello()

'Hello Jose'

Assign a label to the function.  Note that e are not using parentheses here because we are not calling the function hello, instead we are just putting it into the greet variable.

In [10]:
greet = hello

In [11]:
greet

<function __main__.hello>

In [12]:
greet()

'Hello Jose'

This assignment is not attached to the original function:

In [13]:
del hello

In [14]:
hello()

NameError: name 'hello' is not defined

In [15]:
greet()

'Hello Jose'

## Functions within functions
Great! So we've seen how we can treat functions as objects, now lets see how we can define functions inside of other functions:

In [16]:
def hello(name='Jose'):
    print('The hello() function has been executed')
    
    def greet():
        return '\t This is inside the greet() function'
    
    def welcome():
        return "\t This is inside the welcome() function"
    
    print(greet())
    print(welcome())
    print("Now we are back inside the hello() function")

In [17]:
hello()

The hello() function has been executed
	 This is inside the greet() function
	 This is inside the welcome() function
Now we are back inside the hello() function


In [18]:
welcome()

NameError: name 'welcome' is not defined

Note how due to scope, the welcome() function is not defined outside of the hello() function. Now lets learn about returning functions from within functions:
## Returning Functions

In [19]:
def hello(name='Jose'):
    
    def greet():
        return '\t This is inside the greet() function'
    
    def welcome():
        return "\t This is inside the welcome() function"
    
    if name == 'Jose':
        return greet
    else:
        return welcome

In [20]:
x = hello()

Now lets see what function is returned if we set x = hello(), note how the closed parenthesis means that name ahs been defined as Jose.

In [21]:
x

<function __main__.hello.<locals>.greet>

Great! Now we can see how x is pointing to the greet function inside of the hello function.

In [22]:
print(x())

	 This is inside the greet() function


Lets take a quick look at the code again. 

In the if/else clause we are returning greet and welcome, not greet() and welcome(). 

This is because when you put a pair of parentheses after it, the function gets executed; whereas if you don’t put parenthesis after it, then it can be passed around and can be assigned to other variables without executing it.

When we write x = hello(), hello() gets executed and because the name is Jose by default, the function greet is returned. If we change the statement to x = hello(name = "Sam") then the welcome function will be returned. We can also do print hello()() which outputs now you are in the greet() function.

## Functions as Arguments
Now lets see how we can pass functions as arguments into other functions:

In [23]:
def hello():
    return 'Hi Jose!'

def other(func):
    print('Other code would go here')
    print(func())

In [24]:
other(hello)

Other code would go here
Hi Jose!


Great! Note how we can pass the functions as objects and then use them within other functions. Now we can get started with writing our first decorator:

## Creating a Decorator
In the previous example we actually manually created a Decorator. Here we will modify it to make its use case clear:

In [25]:
def new_decorator(func):

    def wrap_func():
        print("Code would be here, before executing the func")

        func()

        print("Code here will execute after the func()")

    return wrap_func

def func_needs_decorator():
    print("This function is in need of a Decorator")

In [26]:
func_needs_decorator()

This function is in need of a Decorator


In [27]:
# Reassign func_needs_decorator
func_needs_decorator = new_decorator(func_needs_decorator)

In [28]:
func_needs_decorator()

Code would be here, before executing the func
This function is in need of a Decorator
Code here will execute after the func()


So what just happened here? A decorator simple wrapped the function and modified its behavior. Now lets understand how we can rewrite this code using the @ symbol, which is what Python uses for Decorators:

In [29]:
@new_decorator
def func_needs_decorator():
    print("This function is in need of a Decorator")

In [30]:
func_needs_decorator()

Code would be here, before executing the func
This function is in need of a Decorator
Code here will execute after the func()


**Great! You've now built a Decorator manually and then saw how we can use the @ symbol in Python to automate this and clean our code. You'll run into Decorators a lot if you begin using Python for Web Development, such as Flask or Django!**

[데코레이터에 대한 고찰 참고](http://mcchae.egloos.com/11030528)